# Text Processing

In [29]:
# We will need a bunch of packages. If they are not installed already, you can install them via
# pip install pandas numpy nltk scikit-learn
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import requests
import json

In [64]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Thomas
[nltk_data]     Weber\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Thomas
[nltk_data]     Weber\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Building a model for Sentiment Analysis

In [16]:
# Load training data
file_path = "iui-05-demo-data.csv"
df = pd.read_csv(file_path)
df = df[['text', 'sentiment']]

In [19]:
# Function for proprocessing the text. See the previous tutorial or the lecture for details
def preprocess_text(text):
    """
    TODO This function should return split a given sentences
    into relevant chunks and remove irrelevant/unimportant words
    """
    if pd.isnull(text):
        return ""

    # Tokenize and convert to lowercase

    tokens = word_tokenize(text.lower())

    stop_words = set(stopwords.words('english'))  # Remove stopwords

    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

    return " ".join(filtered_tokens)

df['cleaned_text'] = df['text'].apply(preprocess_text)


# Map the textual description of the sentiment to a numerical representation
sentiment_map = {'positive': 1, 'negative': 0}

# Filter only positive/negative sentiments
df = df[df['sentiment'].isin(sentiment_map.keys())]

df['sentiment'] = df['sentiment'].map(sentiment_map)

### Training the actual model

In [21]:
# Splitting dataset into training and test sets
X = df['cleaned_text']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [25]:
# For further processing, we need numerical vectors, not text
# Convert text data to numerical vectors using CountVectorizer
vectorizer = CountVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

# Train a logistic regression model
# Feel free to use more complex models to achieve better performance
model = LogisticRegression()
model.fit(X_train_vectors, y_train)

# Make predictions
y_pred = model.predict(X_test_vectors)

In [66]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.863515991036871

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.86      0.86      2375
           1       0.87      0.86      0.87      2534

    accuracy                           0.86      4909
   macro avg       0.86      0.86      0.86      4909
weighted avg       0.86      0.86      0.86      4909



In [75]:
# Testing the model with new data
test_sentences = [
    "I really enjoyed this!",
    "This was terrible.",
]
test_vectors = vectorizer.transform([preprocess_text(sentence) for sentence in test_sentences])
predictions = model.predict(test_vectors)

for sentence, sentiment in zip(test_sentences, predictions):
    print(f"Sentence: {sentence} | Sentiment: {'Positive' if sentiment == 1 else 'Negative'}")


Sentence: I really enjoyed this! | Sentiment: Positive
Sentence: This was terrible. | Sentiment: Negative


## Using LLMs

Instead of a local model, we can also use existing pre-trained models. This does not necessarily need to be a dedicated sentiment model, it can also be a general purpose LLM, e.g. via OpenAI's API.

In this course, we will use an ollama instance which allows hosting openly available models with a defined interface.

In [76]:
# Define Ollama instance configuration
OLLAMA_URL = "https://llm-keyboard.medien.ifi.lmu.de/iui/ollama/api/generate"

# Function to interact with Ollama instance for sentiment analysis
def analyze_sentiment_with_ollama(prompt, model="llama3.1"):
    headers = {
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "temperature": 0.2,
        "top_p": 0.95,
        "top_k": 50,
        "resp_format": "json",
    }
    response = requests.post(OLLAMA_URL, headers=headers, json=payload)

    if response.status_code == 200:
        response_json = response.json()
        return response_json.get("response", "").strip()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None


In [77]:
sentiments = [ ]

for text in test_sentences:
    # prompt = f"Determine the sentiment of the following text as Positive, Negative, or Neutral:\n\n{text}\n"
    prompt = f"Determine the sentiment of the following text as Positive or Negative:\n\n{text}\n\nRespond only using the label for the sentiment."
    sentiment = analyze_sentiment_with_ollama(prompt)
    sentiments.append(sentiment)


for sentence, sentiment in zip(test_sentences, sentiments):
    print(f"Sentence: {sentence} | Sentiment: {sentiment}")

Sentence: I really enjoyed this! | Sentiment: Positive
Sentence: This was terrible. | Sentiment: Negative
